## first model data augmentation

To increase model performace, data augmentation is applied to training data before it is fed into the first model.

Expected result is a reduction in the deviation of training accuracy to test accuracy.

In [1]:
from os.path import join

raw = join('data', 'raw')
processed = join('data', 'processed')

from src.training_env import reset_and_populate

reset_and_populate(raw, processed, [400,0,100])

['data\\processed\\train\\n',
 'data\\processed\\validation\\n',
 'data\\processed\\test\\n',
 'data\\processed\\train\\o',
 'data\\processed\\validation\\o',
 'data\\processed\\test\\o',
 'data\\processed\\train\\x',
 'data\\processed\\validation\\x',
 'data\\processed\\test\\x']

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generator(data_dir, batch_size, datagen):
    full_path = join(processed, data_dir)
    return datagen.flow_from_directory(
        full_path,
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='binary')

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=360,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = create_generator('train', 20, train_datagen)
test_generator = create_generator('test', 10, test_datagen)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [3]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Flatten(input_shape=(32, 32, 3)))
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(3, 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                98336     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 99,491
Trainable params: 99,491
Non-trainable params: 0
_________________________________________________________________


In [4]:
from tensorflow.keras.optimizers import SGD, RMSprop

optimizer = SGD(lr=0.005, momentum=0.9, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [5]:
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from datetime import datetime
from os import mkdir

log_dir = join('logs', 'srp11', datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
mkdir(log_dir)

from src.training_env import reset
reset(log_dir)

callbacks = [ TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1) ]

history = model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=20,
    callbacks=callbacks)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
20/20 [==============================] - 2s 82ms/step - loss: 1.1073 - acc: 0.3300
Epoch 2/20
20/20 [==============================] - 2s 87ms/step - loss: 1.0881 - acc: 0.3925
Epoch 3/20
20/20 [==============================] - 2s 75ms/step - loss: 1.0493 - acc: 0.4375
Epoch 4/20
20/20 [==============================] - 1s 66ms/step - loss: 1.0008 - acc: 0.5550
Epoch 5/20
20/20 [==============================] - 1s 61ms/step - loss: 0.9637 - acc: 0.5600
Epoch 6/20
20/20 [==============================] - 1s 68ms/step - loss: 0.9473 - acc: 0.5775
Epoch 7/20
20/20 [==============================] - 1s 67ms/step - loss: 0.8833 - acc: 0.6000
Epoch 8/20
20/20 [==============================] - 1s 63ms/step - loss: 0.8568 - acc: 0.6700
Epoch 9/20
20/20 [==============================] - 1s 61ms/step - loss: 0.8291 - acc: 0.6525
Epoch 10/20
20/20 [==============================] - 1s 64ms

In [6]:
model.evaluate_generator(test_generator)

[0.6376391430695851, 0.78]

In [7]:
model_path = join('models', 'symbol_classifier', 'first_model_data_augmentation.h5')
model.save(model_path)